# Features definition

<div class="alert alert-block alert-info">
 <span style="font-family:Verdana"> Those are the features we are going to compute throughout this notebook.</span>
</div>


> <span style="font-family:Verdana"> `client_id` : client id  
`n_transactions` : total transactions per client  
`total_amount_per_client` : total amount per client  
`mean_amount_per_transac` : average amount per ticket per client  
`total_product_per_client` : number of products per client  
`mean_product_per_transac` : average number of products per ticket per client  
`unique_product_per_ticket` : number of unique product per ticket per client  
`product_cat` : number of categories the cliend has purchased in  </span>

> <span style="font-family:Verdana"> `n_store` : number of store where the client made a purchase  
`client_type` : male, female, company  
`zipcode` : first two digits of a zipcode  
`client_presence` : number of month since the client's account creation date  
`frequency_orders`: ratio of number of month with a transactions divided by presence of a client account  
`n_month_transac` : number of months with a purchase made by the client  
`recency` : number of month since the client last purchase  </span>

> <span style="font-family:Verdana"> For each time of the day (A), it is the number of transactions made by client during this time :  
B = `['apres_midi','matin', 'midi', 'soir']`  </span>

> <span style="font-family:Verdana"> For each product category (B), it is the number of transactions made by client in one category :  
C = `['Autres', 'Carrelage', 'Chauffage', 'Cuisine',  
     'Dressing', 'Droguerie', 'Décoration', 'Electricité et domotique',  
     'Jardin', 'Luminaires', 'Matériaux de construction', 'Menuiserie',  
     'Outils', 'Peinture', 'Plomberie', 'Quincaillerie', 'Robinetterie',  
     'Salle de bains', 'Terrasse']` </span>

# Basic imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import random

In [2]:
path = '../data'
transactions = pd.read_csv(os.path.join(path,'transactions_light.csv'))
clients = pd.read_csv(os.path.join(path,'clients.csv'))
products = pd.read_csv(os.path.join(path,'products.csv'))

In [3]:
def df_optimized(df, verbose=True, **kwargs):
    """
    Reduces size of dataframe by downcasting numeircal columns
    :param df: input dataframe
    :param verbose: print size reduction if set to True
    :param kwargs:
    :return: df optimized
    """
    df_memory_before = df.memory_usage(deep=True).sum()/1_000_000
    for col in df.columns:
        if 'float' in str(df[col].dtypes) :
            df[col] = pd.to_numeric(df[col],downcast='float')
        elif 'int' in str(df[col].dtypes) :
            df[col] = pd.to_numeric(df[col],downcast='integer')
        if col == 'client_id':
            df[col] = pd.to_numeric(df[col],downcast='integer')
    df_memory_after = df.memory_usage(deep=True).sum()/1_000_000
    memory_saved = round(((df_memory_before - df_memory_after)/ df_memory_before)*100,2)
    
    if verbose:
        print(f"final file memory : {round(df_memory_after,2)} MB")
        print(f"memory saved : {memory_saved}%")
    
    return df


In [4]:
transactions = df_optimized(transactions)
clients = df_optimized(clients)
products = df_optimized(products)

final file memory : 1482.79 MB
memory saved : 28.46%
final file memory : 27.42 MB
memory saved : 5.56%
final file memory : 2.7 MB
memory saved : 4.91%


# Clients

In [5]:
def over_rep(df,over_threshold=0.85,n_values=1):
    """ function to drop columns with over represented value/category 
    :param df: input dataframe
    :param over_threshold: ratio from which we consider it's too high of a representated value/category in the dataset
    :param n_values: number of values, we want to sum and have their % of the whole dataframe
    :return: df
    """
    for col in df.columns:
        if (df[col].value_counts().head(n_values)/df.shape[0]).sum() > over_threshold:
            df = df.drop(col, axis=1)  
    return df

In [6]:
def clean_nan(df,column_threshold=0.5,row_threshold=0.05):
    """ function to drop columns with over represented value/category 
    :param df: input dataframe
    :param na_threshold: ratio from which we consider it's too high of na in the dataset
    :return: df
    """
    for col in df.columns:
        if df[col].isna().sum()/len(df) > column_threshold:
            df = df.drop(col, axis=1)
    for col in df.columns:
        if df[col].isna().sum()/len(df) < row_threshold:
            df = df.dropna()
    for col in df.columns:
        if df[col].isna().sum()/len(df) > row_threshold:
            print(f"a choice has to be made for column {col}")
        
    return df

In [7]:
clients = clean_nan(clients,column_threshold=0.5,row_threshold=0.05)
clients = over_rep(clients,over_threshold=0.85,n_values=1)

In [8]:
clients['client_date'] = pd.to_datetime(clients['client_date'],yearfirst=True)
clients['zipcode'] = clients['zipcode'].astype(str).str.zfill(5).apply(lambda x : x[:2])
clients.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 134289 entries, 0 to 134485
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   client_id    134289 non-null  int32         
 1   client_date  134289 non-null  datetime64[ns]
 2   client_type  134289 non-null  object        
 3   zipcode      134289 non-null  object        
dtypes: datetime64[ns](1), int32(1), object(2)
memory usage: 4.6+ MB


# Products

In [9]:
products.head()

,product_id,product_cat
0,291680,Dressing
1,283879,Matériaux de construction
2,283878,Matériaux de construction
3,283877,Matériaux de construction
4,283876,Matériaux de construction


# Transactions

In [10]:
def clean_data(df):
    #replace infinite values
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    
    # delete rows with NaN values
    df.dropna(axis=0, how='any', inplace=True)
    
    # delete duplicates
    df.drop_duplicates(inplace=True)
    
    #delete rows with 0 in column price_ttc
    df = df[df.price_ttc != 0]
    
    return df

In [11]:
transactions = clean_data(transactions)# transform date data 
transactions['date'] = pd.to_datetime(transactions['date'],yearfirst=True)

## Features

### transactions per client

In [12]:
#total amount spend by client
total_amount_per_client = transactions.groupby('client_id', as_index=False)['price_ttc'].sum()
total_amount_per_client.columns = ['client_id','total_amount_per_client']

#total products ordered by client
total_products_per_client = transactions.groupby('client_id', as_index=False)['n_products'].sum()
total_products_per_client.columns = ['client_id','total_product_per_client']

#number of transations per client
transac_per_client = transactions.groupby('client_id', as_index=False)['ticket_id'].nunique()
transac_per_client.columns = ['client_id','n_transactions']

# number of unique products in transaction per client
unique_product_per_ticket = transactions.groupby(['client_id','ticket_id'], as_index=False)['product_id'].nunique()
unique_product_per_ticket.columns = ['client_id','ticket_id','unique_product_per_ticket']
unique_product_per_ticket = unique_product_per_ticket.groupby('client_id').agg({'unique_product_per_ticket':'mean'})


#number of store where the client made their transactions
n_store_per_client = transactions.groupby('client_id', as_index=False)['store_id'].nunique()
n_store_per_client.columns = ['client_id','n_store']


### time features

In [13]:
transactions['DoW'] = transactions['date'].dt.dayofweek

client_first_purchase = transactions.groupby('client_id', as_index=False)['date'].min()
client_first_purchase.columns = ['client_id','date_first_purchase']

client_last_purchase = transactions.groupby('client_id', as_index=False)['date'].max()
client_last_purchase.columns = ['client_id','date_last_purchase']

# compute each distinct month of purchasing
transactions['year_month'] = pd.to_datetime(transactions['date']).dt.to_period('M')

# compute number of month with transactions to compute a coarse frequency
months_with_transac = transactions.groupby('client_id', as_index=False)['year_month'].nunique().sort_values('year_month',ascending=False)
months_with_transac.columns = ['client_id','n_month_transac']

# define the last date present on the transactions file, 
# it will serve us as referencial to compute the client duration

last_date = transactions.date.max()

clients['client_presence'] =  (last_date - clients['client_date']) / np.timedelta64(24,'h') / 30.5


#### day of the week

In [14]:
dow = {0:'lundi',
      1:'mardi',
      2:'mercredi',
      3:'jeudi',
      4:'vendredi',
      5:'samedi',
      6:'dimanche'}
transactions['DoW'] = transactions.DoW.map(dow)
transaction_DoW = transactions.groupby(['client_id','ticket_id'])['DoW'].value_counts().unstack().fillna(0).reset_index()

# we only want to count one transaction per day per client (and not the number of products bought during this moment)
for col in [i for i in list(transaction_DoW.columns) if i != 'ticket_id' and i != 'client_id']:
    transaction_DoW[col] = transaction_DoW[col].apply(lambda x: 1 if x >= 1 else 0)

# we want to have the total of transactions made by day by the client
transaction_DoW = transaction_DoW.groupby('client_id').agg({'lundi':'sum',
                                               'mardi':'sum',
                                               'mercredi':'sum',
                                               'jeudi':'sum',
                                                'vendredi':'sum',
                                                'samedi':'sum',
                                                'dimanche':'sum'}).reset_index()





#### moment of purchasing

In [15]:
############################################################################################
#  We want to compute how many times a client has purchased during each moment of the day  #
############################################################################################

# transform hour into a categorical feature
nuit = [i for i in range(5)] + [24]
matin = [i for i in range(6,12)]
midi = [i for i in range(12,15)]
apres_midi = [i for i in range(15,19)]
soir = [i for i in range(19,23)]
jour_name = ['nuit','matin','midi','apres_midi','soir']
jour_cat = [nuit,matin,midi,apres_midi,soir]
hour_cat = {i : name for cat,name in zip(jour_cat,jour_name) for i in cat}

# allows at each transaction a moment of day when the purchase occurs
transactions['hour_cat'] = transactions.hour.map(hour_cat)

# create a column by moment 
transaction_hour_cat = transactions.groupby(['client_id','ticket_id'])['hour_cat'].value_counts().unstack().fillna(0).reset_index()

# reduce size of thoses columns
for col in transaction_hour_cat.columns:
    if 'float' in str(transaction_hour_cat[col].dtypes) :
        transaction_hour_cat[col] = pd.to_numeric(transaction_hour_cat[col],downcast='integer')
        
# we only want to count one transaction per moment per client (and not the number of products bought during this moment)
for col in [i for i in list(transaction_hour_cat.columns) if i != 'ticket_id' and i != 'client_id']:
    transaction_hour_cat[col] = transaction_hour_cat[col].apply(lambda x: 1 if x >= 1 else 0)

# we want to have the total of transactions made by moment by the client
transaction_hour_cat = transaction_hour_cat.groupby('client_id').agg({'apres_midi':'sum',
                                               'matin':'sum',
                                               'midi':'sum',
                                               'soir':'sum'}).reset_index()



### cat products

In [16]:
###############################################################################################
#  We want to compute how many times a client has purchased products in a specific category  ##
###############################################################################################

temp = transactions[['client_id','ticket_id','product_id']].merge(products, on='product_id')
tp = temp.groupby(['client_id','ticket_id'], as_index=False)['product_cat'].nunique()
cat_per_trans = tp.groupby('client_id', as_index=False).agg({'product_cat':'mean'})

products_order_per_cat = temp.groupby(['client_id'])['product_cat'].value_counts().unstack().fillna(0)

<div class="alert alert-block alert-warning">
<span style="font-family:Verdana"> Here we want to highlight the number of products bought by category vs in transaction_hour_cat where the focus is on the number of transactions made.</span>
</div>

# Merge tables

In [17]:
#merge with amounts tables
client_base = transac_per_client.merge(total_amount_per_client, how='inner')
client_base['mean_amount_per_transac'] = client_base['total_amount_per_client'] / client_base['n_transactions']

#merge with products tables
client_base = client_base.merge(total_products_per_client, how='inner')
client_base['mean_product_per_transac'] = client_base['total_product_per_client'] / client_base['n_transactions']

client_base = client_base.merge(unique_product_per_ticket, on='client_id', how='inner')
client_base = client_base.merge(cat_per_trans, on='client_id', how='inner')


# merge with time features
client_base = client_base.merge(n_store_per_client, how='inner')\
                         .merge(client_first_purchase, how='inner')\
                         .merge(client_last_purchase, how='inner')\
                         .merge(months_with_transac, how='inner')


client_base = client_base.merge(clients, on='client_id', how='inner')

# compute frequency ratio
client_base['frequency_orders'] = round(client_base['n_month_transac'] / client_base['client_presence'],4)

# compute recency
client_base['recency'] = (last_date - client_base['date_last_purchase']) / np.timedelta64(24,'h') / 30.5
client_base = client_base.drop(['client_date','date_first_purchase','date_last_purchase'], axis=1)

# add time of the day columns
client_base = client_base.merge(transaction_hour_cat, on='client_id', how='inner')

# add day of the week columns
client_base = client_base.merge(transaction_DoW, on='client_id', how='inner')

# add category columns
client_base = client_base.merge(products_order_per_cat, on='client_id', how='inner')


In [18]:
############################################################
#  reduce size of the dataframe and save it for modelling  #
############################################################

X = df_optimized(client_base)
X.to_csv(os.path.join(path,'X_full.csv'), index=False)

final file memory : 36.64 MB
memory saved : 42.78%


# Checks

## check with client number

In [19]:
test = transactions[transactions['client_id']==0].sort_values(by='ticket_id')
test.merge(products, on='product_id').sort_values(by='ticket_id')

,ticket_id,product_id,store_id,client_id,date,hour,n_products,price_ttc,DoW,year_month,hour_cat,product_cat
0,1813374,411254,51,0,2019-10-18,10,11,48.730000,vendredi,2019-10,matin,Plomberie
1,2424830,443448,62,0,2019-04-01,12,2,267.040009,lundi,2019-04,midi,Salle de bains
2,2424830,443496,62,0,2019-04-01,12,2,142.199997,lundi,2019-04,midi,Salle de bains
3,2424830,178185,62,0,2019-04-01,12,1,15.270000,lundi,2019-04,midi,Outils
4,2424830,71501,62,0,2019-04-01,12,1,15.290000,lundi,2019-04,midi,Quincaillerie
...,...,...,...,...,...,...,...,...,...,...,...,...
237,5848177,671191,62,0,2020-04-02,13,1,3.570000,jeudi,2020-04,midi,Matériaux de construction
73,5848177,812727,62,0,2020-04-02,13,4,57.279999,jeudi,2020-04,midi,Jardin
238,5848177,641276,62,0,2020-04-02,13,1,1.920000,jeudi,2020-04,midi,Droguerie
239,5848816,8,62,0,2018-10-29,8,1,5.070000,lundi,2018-10,matin,Autres


## check with transac number

In [20]:
transactions[transactions['ticket_id']==5882797]

,ticket_id,product_id,store_id,client_id,date,hour,n_products,price_ttc,DoW,year_month,hour_cat
6436193,5882797,823904,255,1471152,2019-05-29,14,12,22.920000,mercredi,2019-05,midi
6436213,5882797,828656,255,1471152,2019-05-29,14,6,47.040001,mercredi,2019-05,midi
6436220,5882797,828698,255,1471152,2019-05-29,14,7,20.860001,mercredi,2019-05,midi
6436228,5882797,861298,255,1471152,2019-05-29,14,81,85.860001,mercredi,2019-05,midi
6440977,5882797,861312,255,1471152,2019-05-29,14,7,57.330002,mercredi,2019-05,midi
6447519,5882797,82534,255,1471152,2019-05-29,14,2,18.059999,mercredi,2019-05,midi
6449351,5882797,823905,255,1471152,2019-05-29,14,7,34.650002,mercredi,2019-05,midi
6449352,5882797,824015,255,1471152,2019-05-29,14,11,81.510002,mercredi,2019-05,midi
